In [117]:
using LinearAlgebra 
using Printf
using PrettyTables
#Pregunta 1: implementar el metodo de Evans


function es_penta(a)
    #=
    Verifica si la matriz es pentadiagonal
    Entrada: matriz a evaluar
    Salida: Retorna un valor booleano indicando si es pentadiagonal
    =#
    flag = 0;#flag para terminar el ciclo
    m = size(a)[1];# columnas de la matriz
    n = size(a)[2];# n de la matriz
      
    es = true;#output
    
    if m >= 4
        for i in 1:m # recorrer filas

            for j in 1:n #recorrer columnas
                if abs(i-j) > 2
                    if a[i,j] != 0
                        es = false;
                        flag = 1;
                        break
                    else
                        es = true;
                    end
                end
            end

          if flag == 1 #terminar loop
            break
          end
        end
        
        
    else
        es = false;
    end
    
    return es;
end








function det_penta(a)
    #=
    % calcula el determinante de una matriz pentadiagonal
    % Entrada: matriz a evaluar
    % Salida: determiante de la matriz
    =#
    
    if es_penta(a) == false # verifica que sea pentadiagonal
        println("La matriz no es pentadiagonal!!!");
    else
        
        m = size(a)[1];#filas de la matriz
        
        #calculo de variables
        p1 = a[1,1];
        p2 = a[2,2]*p1 - a[2,1]*a[1,2];
        r2 = a[1,2];
        s2 = a[2,1];
        p3 = a[3,3]*p2 - a[3,2]*a[2,3]*p1 - a[3,1]*a[1,3]*a[2,2] + a[3,1]*a[2,3]*r2 + a[3,2]*a[1,3]*s2;
        r3 = a[3,2]*p1 - a[1,3]*s2;
        s3 = a[3,2]*p1 - a[3,1]*r2;
        p4 = a[4,4]*p3 - a[4,3]*a[3,4]*p2 - a[4,2]*a[2,4]*(a[3,3]*p1 - a[3,1]*a[1,3]) + a[4,2]*a[3,4]*r3 + a[4,3]*a[2,4]*s3;
        
        
        p = Vector{Float64}();
        append!(p,p1);
        append!(p,p2);
        append!(p,p3);
        append!(p,p4);
        
        r = Vector{Float64}();
        append!(r,0);
        append!(r,r2);
        append!(r,r3);
        
        s = Vector{Float64}();
        append!(s,0);
        append!(s,s2);
        append!(s,s3);

        for i in 5:m
            ri_1 = a[i-2,i-1]*p[i-3] - a[i-3,i-1]*s[i-2];
            append!(r,ri_1);

            si_1 = a[i-1,i-2]*p[i-3] - a[i-1,i-3]*r[i-2];
            append!(s,si_1);

            Pi = a[i,i]*p[i-1] - a[i,i-1]*a[i-1,i]*p[i-2] - a[i,i-2]*a[i-2,i]*(a[i-1,i-1]*p[i-3]
                - a[i-1,i-3]*a[i-3,i-1]*p[i-4]) + a[i,i-2]*a[i-1,i]*r[i-1] + a[i,i-1]*a[i-2,i]*s[i-1];
            append!(p,Pi);
        end

        return p[end];
        
    end
end

#det_penta(C)
#============================================================================================#

#parte 2


function ptrans_1(A,Y)
    #=
    Resuelve el sistema de ecuaciones de matrices pentadiagonales
    Entrada: Matriz pentadiagonal, vector de coeficiente
    Salida: Vector solución del sistema 
    =#

    if !es_penta(A)#revisa si es pentadiagonal
        println("La matriz ingresada no es pentadiagonal!!");
    else
        if det_penta(A) == 0 #revisa si es inversible con el determinante
            println("El sistema no tiene solucion!!");
        else
            n = size(A)[1];#tamaño de la matriz
            
            #se sacan las diagonales
            a = diag(A,1);
            b = diag(A,2);
            c = diag(A,-1);
            d = diag(A,0);
            e = diag(A,-2);
            
            #Determinar u1,aplha1,beta1,z1
            u1 = d[1];
            alpha1 = a[1]/u1;
            beta1 = b[1]/u1;
            z1 = Y[1]/u1;
            
            #Determinar gamma2,u2,aplha2,beta2,z2
            gamma2 = c[2];
            u2 = d[2] - alpha1*gamma2;
            alpha2 = (a[2] - beta1*gamma2)/u2;
            beta2 = b[2]/u2;
            z2 = (Y[2]-z1*gamma2)/u2;    
            
           
            
            #añadir en los vectores respectivamente
            u = Vector{Float64}();
            append!(u,u1);
            append!(u,u2);
            
            alpha = Vector{Float64}();
            append!(alpha,alpha1);
            append!(alpha,alpha2);
            
            beta = Vector{Float64}();
            append!(beta,beta1);
            append!(beta,beta2);
            
            z = Vector{Float64}();
            append!(z,z1);
            append!(z,z2);
            
            gamma = Vector{Float64}();
            append!(gamma,0);
            append!(gamma,gamma2);
            
            #Calculo de gammai,ui,alphai,betai,zi de i=3 a i = n-2
            for i = 3: n-2
                gammai = c[i]-alpha[i-2]*e[i];
                append!(gamma,gammai);
                
                ui = d[i] - beta[i-2]*e[i] - alpha[i-1]*gamma[i];
                append!(u,ui);
                
                alphai = (a[i] - beta[i-1]*gamma[i])/ui;
                append!(alpha,alphai);
                
                betai = b[i]/u[i];
                append!(beta,betai);
                
                zi =(Y[i] - z[i-2]*e[i] - z[i-1]*gamma[i])/u[i];
                append!(z,zi);
            end
            
           
            #calculo de los 2 ultimos valores de cada vector
            gamman_1 = c[n-1] - alpha[n-3]*e[n-2];
            append!(gamma,gamman_1);
            
            un_1 = d[n-1] - beta[n-3]*e[n-2] - alpha[n-2]*gamman_1;
            append!(u,un_1);
            
            alphan_1 = (a[n-1] - beta[n-2]*gamman_1)/u[n-1];
            append!(alpha,alphan_1);
            

            gamman = c[n-1] - alpha[n-2]*e[n-2];
            append!(gamma,gamman);
            
            un = d[n] - beta[n-2]*e[n-2] - alpha[n-1]*gamma[n-1];
            append!(u,un);
            
            zn_1 = (Y[n-1] - z[n-3]*e[n-2] - z[n-2]*gamma[n-1])/u[n-1];
            append!(z,zn_1);
            
            zn = (Y[n] - z[n-2]*e[n-2] - z[n-1]*gamma[n])/u[n];
            append!(z,zn);
            
           
            X = zeros(n,1);#vector solución
    
          
            xn = z[n];
            xn_1 = z[n-1]-alpha[n-1]*xn;
            
            X[n] = xn;

            X[n-1]= xn_1;
            
            
            # se calcula el valor del vector solución
            for i in n-2:-1:1
               
                xi = z[i] - alpha[i]*X[i+1] - beta[i]*X[i+2];
                X[i] = xi;
            end
           
            
            return X;
        end
        
        
        
    end
    
    
end


    





function create_matrizN1(n)
    
    M = zeros(Int64,n,n);
    M[1,1] = 9;
    
    #diagonal central
    for i = 2:n
        M[i,i] = 6;
    end
    M[n-1,n-1] = 5;
    M[n,n] = 1;
    
    
    #diagonal derecha 1
    j = 2;
    for i = 1:n-1
        M[i,j] = -4;
        j += 1;
    end
    
    M[n-1,n] = -2;
    
    #diagonal izquierda 1
    j = 1;
    for i = 2:n-1
        M[i,j] = -4;
        j += 1;
    end
    
    
    M[n,n-1] = -2;
    
    
    #diagonal derecha 2
    j = 3;
    for i = 1:n-2
        M[i,j] = 1;
        j += 1;
    end
    
    #diagonal izquierda 2
    j = 1;
    for i = 3:n
        M[i,j] = 1;
        j += 1;
    end
    
    return M;
end

function create_y1(n)
    X = zeros(n,1);
    X[1] = 6;
    X[2] = -1;
    return X
end


A = create_matrizN1(500);
A2 = create_matrizN1(5000);
A3 = create_matrizN1(10000);


b = create_y1(500);
b2 = create_y1(5000);
b3 = create_y1(10000);



X = ptrans_1(A,b);
tiempo1 = @elapsed ptrans_1(A,b);

X2 = ptrans_1(A2,b2);
tiempo2 = @elapsed ptrans_1(A2,b2);

X3 = ptrans_1(A3,b3);
tiempo3 = @elapsed ptrans_1(A3,b3);





N1 = norm(A*X-b);
N2 = norm(A2*X2-b2);
N3 = norm(A3*X3-b3);





data = [500       N1      tiempo1;
        5000      N2      tiempo2;
        10000     N3      tiempo3;
        50000     "-"      "-";]
         
header = (["n","||A*x - b||","Tiempo"]);
pretty_table(data; header = header);

┌───────┬─────────────┬───────────┐
│     n │ ||A*x - b|| │    Tiempo │
├───────┼─────────────┼───────────┤
│   500 │  5.42994e-8 │ 0.0008553 │
│  5000 │ 5.37311e-11 │  0.308218 │
│ 10000 │ 6.70918e-12 │   1.66449 │
│ 50000 │           - │         - │
└───────┴─────────────┴───────────┘
